In [1]:
from transformers import pipeline, TFAutoModelForSequenceClassification, AutoTokenizer
from typing import Dict
import tensorflow as tf

class TransformerModel():
    def __init__(self):
        # Emotion detection model (uses TF internally if available)
        self.emotion_model = pipeline(
            'text-classification',
            model='SamLowe/roberta-base-go_emotions',
            return_all_scores=True,
            framework="tf"  # Force TensorFlow
        )
        
        # Sarcasm detection model (TensorFlow version)
        self.sarcasm_tokenizer = AutoTokenizer.from_pretrained("MohamedGalal/marbert-sarcasm-detector")
        self.sarcasm_model = TFAutoModelForSequenceClassification.from_pretrained(
            "MohamedGalal/marbert-sarcasm-detector"
        )

        # Sentiment analysis model (TensorFlow)
        self.sentiment_model = pipeline(
            "sentiment-analysis",
            model="cardiffnlp/twitter-roberta-base-sentiment-latest",
            framework="tf"
        )

        # Brand perception model (TensorFlow)
        self.perception_model = pipeline(
            "zero-shot-classification",
            model="facebook/bart-large-mnli",
            framework="tf"
        )

    def detect_emotions(self, text: str) -> Dict:
        results = self.emotion_model(text)[0]
        return {item['label']: item['score'] for item in results}
    
    def detect_sarcasm(self, text: str) -> Dict:
        inputs = self.sarcasm_tokenizer(text, return_tensors="tf", truncation=True)
        outputs = self.sarcasm_model(**inputs)
        probs = tf.nn.softmax(outputs.logits, axis=1)
        return {
            'sarcastic': float(probs[0][1]),
            'literal': float(probs[0][0])
        }
    
    def analyze_sentiment(self, text: str) -> Dict:
        result = self.sentiment_model(text)[0]
        return {'label': result['label'], 'score': result['score']}
    
    def assess_brand_perception(self, text: str, brand_name: str) -> Dict:
        candidate_labels = [
            "positive", "negative", "trustworthy", 
            "innovative", "expensive", "good value",
            "reliable", "poor quality", "luxury",
            "outdated", "trendy", "ethical"
        ]
        result = self.perception_model(text, candidate_labels, multi_label=True)
        return {
            'brand': brand_name,
            'perception_scores': dict(zip(result['labels'], result['scores']))
        }
    
    def full_analysis(self, text: str, brand_name: str) -> Dict:
        return {
            "text": text,
            "emotions": self.detect_emotions(text),
            "sarcasm": self.detect_sarcasm(text),
            "sentiment": self.analyze_sentiment(text),
            "brand_perception": self.assess_brand_perception(text, brand_name)
        }

In [2]:
model = TransformerModel()
text = "The Apple iPhone has a stunning display, a sleek design, and an innovative multitouch user interface. Its Safari browser makes for a superb Web surfing experience, and it offers easy-to-use apps"
brand_name = "Apple"
print(model.full_analysis(text, brand_name))
#Example Usage

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFRobertaForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.
c:\Users\monk\AppData\Local\Programs\Python\Python38\lib\site-packages\transforme

{'text': 'The Apple iPhone has a stunning display, a sleek design, and an innovative multitouch user interface. Its Safari browser makes for a superb Web surfing experience, and it offers easy-to-use apps', 'emotions': {'admiration': 0.8941521048545837, 'amusement': 0.000762377108912915, 'anger': 0.0011404341785237193, 'annoyance': 0.004643904510885477, 'approval': 0.11654303222894669, 'caring': 0.0012699570506811142, 'confusion': 0.0012355491053313017, 'curiosity': 0.0015600243350490928, 'desire': 0.0012235111789777875, 'disappointment': 0.002970066387206316, 'disapproval': 0.00441938079893589, 'disgust': 0.0007426306256093085, 'embarrassment': 0.00027446882450021803, 'excitement': 0.004804884549230337, 'fear': 0.00037267623702064157, 'gratitude': 0.0035960860550403595, 'grief': 0.0002486078883521259, 'joy': 0.002622207859531045, 'love': 0.0026080841198563576, 'nervousness': 0.00011329929111525416, 'optimism': 0.004790074657648802, 'pride': 0.0031123158987611532, 'realization': 0.0090

In [18]:
import tweepy
from dotenv import load_dotenv
import os
load_dotenv()

#Twitter Api Credential

BEARER_TOKEN= os.getenv("BEARER_TOKEN")
print(BEARER_TOKEN)
client = tweepy.Client(bearer_token=BEARER_TOKEN, wait_on_rate_limit=True)

AAAAAAAAAAAAAAAAAAAAACGH0gEAAAAAyLnip2yy9zt5pIZcxIvWHhSMVlE%3DR2P0v6R2SOWKcOUJNMwJPgZzuUMEep9qw22JGr1gOcpI3hx1J5


In [19]:
query = "OpenAI OR ChatGPT -is:retweet lang:en"

# --- Set how many tweets you want (max 100 per request, 500 per 15 min on free tier) ---
max_results = 20
tweet_data = []

# --- Fetch tweets ---
response = client.search_recent_tweets(query=query, max_results=max_results,
                                       tweet_fields=["created_at", "author_id", "lang", "public_metrics"])

if response.data:
    for tweet in response.data:
        tweet_data.append({
            "id": tweet.id,
            "author_id": tweet.author_id,
            "created_at": tweet.created_at,
            "text": tweet.text,
            "retweets": tweet.public_metrics['retweet_count'],
            "likes": tweet.public_metrics['like_count'],
        })

In [20]:
print(tweet_data)

[{'id': 1912134409420931565, 'author_id': 2931331834, 'created_at': datetime.datetime(2025, 4, 15, 13, 22, 23, tzinfo=datetime.timezone.utc), 'text': 'RT @zinssmeister: Great summary of OpenAI Dev Day by @danshipper https://t.co/UVT8kykjEV', 'retweets': 2, 'likes': 0}, {'id': 1912134398163513593, 'author_id': 1641695097157738496, 'created_at': datetime.datetime(2025, 4, 15, 13, 22, 20, tzinfo=datetime.timezone.utc), 'text': 'RT @artemp_official: OpenAI is phasing out GPT-4.5 — their biggest AI model ever.\nThe future is shifting fast. Adapt or get left behind. ht…', 'retweets': 4, 'likes': 0}, {'id': 1912134369478660577, 'author_id': 1366210521238818818, 'created_at': datetime.datetime(2025, 4, 15, 13, 22, 14, tzinfo=datetime.timezone.utc), 'text': "RT @rowancheung: TODAY'S AI NEWS: OpenAI just dropped three GPT-4.1 models for devs—API-only and packing a jaw-dropping 1M-token context wi…", 'retweets': 45, 'likes': 0}, {'id': 1912134365582143722, 'author_id': 1655362768319533056, 'creat

In [21]:
import pandas as pd 
df = pd.DataFrame(tweet_data)
df.to_csv("tweets_data.csv", index=False)
print("Tweets saved to tweets_data.csv")

Tweets saved to tweets_data.csv


In [23]:
for item in tweet_data:
    text = item['text']
    brand_name = "OpenAI"
    analysis = model.full_analysis(text, brand_name)
    print(analysis)
    

{'text': 'RT @zinssmeister: Great summary of OpenAI Dev Day by @danshipper https://t.co/UVT8kykjEV', 'emotions': {'admiration': 0.4936091899871826, 'amusement': 0.0014209151268005371, 'anger': 0.0007239904371090233, 'annoyance': 0.004931050352752209, 'approval': 0.08874694257974625, 'caring': 0.0006224209791980684, 'confusion': 0.0013144491240382195, 'curiosity': 0.0011651625391095877, 'desire': 0.0005814520409330726, 'disappointment': 0.0023852966260164976, 'disapproval': 0.005765831097960472, 'disgust': 0.000831448589451611, 'embarrassment': 0.000237070897128433, 'excitement': 0.002339666010811925, 'fear': 0.0001593639317434281, 'gratitude': 0.012419866397976875, 'grief': 0.00012858463742304593, 'joy': 0.00154229742474854, 'love': 0.0012537463335320354, 'nervousness': 5.003037949791178e-05, 'optimism': 0.003088885685428977, 'pride': 0.0013629196910187602, 'realization': 0.007226433139294386, 'relief': 0.0004494129098020494, 'remorse': 0.0001316031557507813, 'sadness': 0.0004376461729

KeyboardInterrupt: 